In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import scipy.sparse as sp
import scipy.sparse.linalg as linalg
from matplotlib import cm
from matplotlib import animation


from matplotlib import rc


# Define font for figures
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

golden = (1 + 5 ** 0.5) / 2 #Define golden ratio for plots

%matplotlib inline


In [ ]:
def init_1D():
    "Initializes the 1d animation with a clean slate"
    line.set_data([], [])
    #V_x.set_data([], [])
    return line#, V_x,


# animation function.  This is called sequentially for every frame.
def animate_1D(i):
    "sets the frames for the animation in 1d"
    y = np.absolute(u_plot[:, i])**2
    line.set_data(x, y)
    ttl.set_text(r'z = ' + "%.2f" % (h * i * 1000) + r'm')
    #V_x.set_data(x, V)
    return line#, V_x,


def animation_1D():
    "calls the animation function in 1d"
    anim = animation.FuncAnimation(fig, animate_1D, init_func=init_1D,
           frames=animation_frames, save_count=anim_count, interval=5, blit=False, repeat=False)
    return anim

def save_anim(file, title):
    "saves the animation with a desired title"
    Writer = animation.writers['ffmpeg']
    writer = Writer(fps=25, metadata=dict(artist='Me'), bitrate=1800)
    file.save(title + '.mp4', writer=writer)

In [ ]:
#Define constants
wl = 0.63e-6 #[m] wavelength
N = 2048 #grid points
L = 50e-3 #physical size
z = 1 # [m] propagation length
ap_size = 1.5e-3 #[m] apperture radius
xmin = -2.5e-3
xmax = 2.5e-3

#Define space and frequency domain
dx = L/N 
x_four = np.arange(-L/2, L/2, dx)
df = 1/L
fx = np.arange(-0.5/dx, 0.5/dx, df)
[XX, YY] = np.meshgrid(x_four, x_four)
[FX, FY] = np.meshgrid(fx, fx)

#Make and show aperture
Aperture = np.squeeze(np.array([XX**2 + YY**2 < (ap_size)**2])) #define circular aperture
# ### Uncomment for plot ### #
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot_surface(XX, YY, Aperture)

In [ ]:
#Define shift necessary for x and y direction due to figure being centered
[shiftx, shifty] = np.exp(-2*np.pi*1j*np.array([FX, FY])*L/2) * dx

#Define the Fresnel Propagator
fresnel_prop = np.exp(-1j*np.pi*wl*z*(FX**2 + FY**2)) * np.exp(2*np.pi*1j*z/wl)

#Caclulate field at z = 1 by IFT(FT(Aperture)*propagator)
Aperture_trans = np.fft.fftshift(np.fft.fft2(Aperture)) * shiftx * shifty
Aperture_propagated = fresnel_prop*Aperture_trans
field_image_plane = np.fft.ifftshift(np.fft.ifft2(Aperture_propagated)) / (shiftx*shifty)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)

cont_field = axes[0].contourf(XX, YY, np.abs(field_image_plane)**2, cmap = cm.RdBu_r)
cbar = fig.colorbar(cont_field, ax = axes[0])
axes[0].locator_params(nbins = 4)
axes[0].set_title('Field intensity at $z = 1$ m')
axes[0].set_xlim([xmin, xmax])
axes[0].set_ylim([xmin, xmax])

axes[1].plot(x_four, np.abs(field_image_plane[int(0.5*N), :])**2)

axes2 = plt.gca()
axes2.set_xlim([xmin, xmax])
axes2.set_ylim([0, 2])
axes[1].locator_params(nbins = 4)
axes[1].set_title(r'profile at $x= 0$ ')

# # Uncomment for addition 3d plot
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# threed_plot = ax.plot_surface(XX, YY, np.abs(field_image_plane)**2, cmap = cm.RdBu_r)
# ax.set_title('Field inentisty at $z=1$ m')

#ax.set_xlim([xmin/10,xmax/10])
#ax.set_ylim([xmin/10,xmax/10])
#ax.set_zlim([0, 2])

# Finite difference method

Find the field at $z = 1$m by making use of a finite difference method. A Crank Nicholson implicit integration scheme is used to solve the field at $z + \Delta z$. Dirichlet boundary conditions with the forcing term equal to 0 are used, which is a good approximation of free space since the width of the solution is far less than the domain it is solved on.

In [ ]:
# Set parameters
wl = 0.63e-6 #[m] wavelength
L = 50e-3 #physical size
z = 1 # [m] propagation length
ap_size = 1.5e-3

h = 1e-5 # delta z size
xmin = -L/2
xmax = L/2
nodes = 10000
k = 2*np.pi/wl

a = (xmax - xmin) / (nodes-1)  # space between nodes
x = np.linspace(xmin, xmax, nodes)
zsteps = int(1/h)

# initial wave function
u = np.array([abs(x) < ap_size], dtype = np.cfloat)

# Make left hand side matrix
ai = 1/h - 1/(2*1j*k*a**2)
bi = 1/(4*1j*k*a**2)

A = sp.diags([bi, ai, bi], [-1, 0 ,1],shape=(nodes, nodes)).tocsc()

# Make right hand side vector
ci = 1/h + 1/(2*1j*k*a**2)
di = -1/(4*1j*k*a**2)
B = sp.diags([di, ci, di], [-1, 0, 1],shape=(nodes, nodes)).tocsc()

## Attempt at ILU decomposition as preconditioner for bicgstab algorithm.
lu = linalg.spilu(A)
MU = lu.U
ML = lu.L
M = ML.dot(MU)


anim_constant = 1000
anim_count = 0
animation_frames = int((zsteps)/anim_constant)

u_plot = np.zeros(shape=(nodes, animation_frames), dtype=np.cfloat)

for t in range(zsteps):
    b = B.dot(u.T)
    u = linalg.bicgstab(A, b, None, 1e-5, None, None, M, None)[0]
    if (t+1) % anim_constant == 0:
        u_plot[:,anim_count] = u
        anim_count += 1


In [ ]:
# Show animations
fig = plt.figure()
ax = plt.axes(xlim=(xmin/10, xmax/10), ylim=(0, 2))
ax.set_xlabel(r'$r$ [m]')
ax.set_ylabel(r'$|u|^2$ [J]')
line, = ax.plot([], [], lw=2)
ttl = ax.text(.5, 1.05, '', transform = ax.transAxes, va='center')

# call the animator.
anim_sq_well = animation_1D()

plt.show()

In [ ]:
save_anim(anim_sq_well,'Intensity_propagation')

In [ ]:
plotmin = xmin/10
plotmax = xmax/10

dz = -20000

for ii in range(3,4):  
    distance = int(zsteps + dz + (ii-10)*500)
    fig = plt.figure(figsize=plt.figaspect(1/golden))
    ax = fig.add_subplot(111)
    ax.plot(x, np.abs(u_plot[:, distance])**2, label = r'Finite Difference Method' )
    ax.plot(x_four, np.abs(field_image_plane[int(0.5*N)+1,:])**2, label = r'Fourier propagation')
    ax.set_title(r'FD at $z = $' + str(distance * h) + r'm, FT at $z=1$ m')
    axes = plt.gca()
    axes.set_xlim([plotmin, plotmax])
    axes.set_ylim([0,2])
    legend = ax.legend(loc='upper right', shadow=True, fontsize = 9)

In [ ]:
fig.savefig('Circular_aperture_interference_fd_ft_propagated.pdf', bbox_inches='tight', pad_inches=0.1)

# Check dz and dx independence

In [ ]:
# Set parameters
wl = 0.63e-6 #[m] wavelength
L = 50e-3 #physical size
z = 1 # [m] propagation length
ap_size = 1.5e-3

h = 1e-4 # delta z size
xmin = -L/2
xmax = L/2
nodes = 10000
k = 2*np.pi/wl

a = (xmax - xmin) / (nodes-1)  # space between nodes
x4 = np.linspace(xmin, xmax, nodes)
zsteps = int(1/h)

# initial wave function
ue4 = np.array([abs(x4) < ap_size], dtype = np.cfloat)

# Make left hand side matrix
ai = 1/h - 1/(2*1j*k*a**2)
bi = 1/(4*1j*k*a**2)

A = sp.diags([bi, ai, bi], [-1, 0 ,1],shape=(nodes, nodes)).tocsc()

# Make right hand side vector
ci = 1/h + 1/(2*1j*k*a**2)
di = -1/(4*1j*k*a**2)
B = sp.diags([di, ci, di], [-1, 0, 1],shape=(nodes, nodes)).tocsc()

## Attempt at ILU decomposition as preconditioner for bicgstab algorithm.
lu = linalg.spilu(A)
MU = lu.U
ML = lu.L
M = ML.dot(MU)

for t in range(zsteps):
    b = B.dot(ue4.T)
    ue4 = linalg.bicgstab(A, b, None, 1e-5, None, None, M, None)[0]


In [ ]:
# Set parameters
wl = 0.63e-6 #[m] wavelength
L = 50e-3 #physical size
z = 1 # [m] propagation length
ap_size = 1.5e-3

h = 1e-4 # delta z size
xmin = -L/2
xmax = L/2
nodes = 20000
k = 2*np.pi/wl

a = (xmax - xmin) / (nodes-1)  # space between nodes
x5 = np.linspace(xmin, xmax, nodes)
zsteps = int(1/h)

# initial wave function
ue5 = np.array([abs(x5) < ap_size], dtype = np.cfloat)

# Make left hand side matrix
ai = 1/h - 1/(2*1j*k*a**2)
bi = 1/(4*1j*k*a**2)

A = sp.diags([bi, ai, bi], [-1, 0 ,1],shape=(nodes, nodes)).tocsc()

# Make right hand side vector
ci = 1/h + 1/(2*1j*k*a**2)
di = -1/(4*1j*k*a**2)
B = sp.diags([di, ci, di], [-1, 0, 1],shape=(nodes, nodes)).tocsc()

## Attempt at ILU decomposition as preconditioner for bicgstab algorithm.
lu = linalg.spilu(A)
MU = lu.U
ML = lu.L
M = ML.dot(MU)

for t in range(zsteps):
    b = B.dot(ue5.T)
    ue5 = linalg.bicgstab(A, b, None, 1e-5, None, None, M, None)[0]


In [ ]:
plotmin = xmin/10
plotmax = xmax/10
fig = plt.figure(figsize=plt.figaspect(1/golden))
ax = fig.add_subplot(111)
ax.plot(x4, np.abs(ue4)**2, label = r'Finite Difference Method with $n = 10 000$' )
ax.plot(x5, np.abs(ue5)**2, label = r'Finite Difference Method with $n = 20 000$' )
#ax.plot(x5, np.abs(field_image_plane[int(0.5*N)+1,:])**2, label = r'Fourier propagation')
#ax.set_title(r'FD at $z = $' + str(distance * h) + r'm, FT at $z=1$ m')
axes = plt.gca()
axes.set_xlim([plotmin, plotmax])
axes.set_ylim([0,2])
legend = ax.legend(loc='upper right', shadow=True, fontsize = 9)

In [ ]:
fig.savefig('test.pdf', bbox_inches='tight', pad_inches=0.1)